Link dataset: https://www.kaggle.com/datasets/hsankesara/flickr-image-dataset

Sau khi download về thì giải nén

# Đọc file dữ liệu flickr30K_images

In [181]:
import numpy as numpy
import pandas as pd
from PIL import Image
import os
import shutil

In [150]:
image_data_df = pd.read_csv('./results.csv').dropna(axis=1)
image_data_df = image_data_df.applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)
image_data_df = image_data_df.iloc[:, 0].str.split('|', expand=True)
image_data_df.columns = ['image_name', 'comment_number', 'comment']
image_data_df ['comment'] = image_data_df ['comment'].apply(lambda x: '' if pd.isna(x) else x)
image_data_df.info()

C:\Users\phuon\AppData\Local\Temp\ipykernel_28360\1774336321.py:1: DtypeWarning: Columns (7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  image_data_df = pd.read_csv('./results.csv').dropna(axis=1)
C:\Users\phuon\AppData\Local\Temp\ipykernel_28360\1774336321.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  image_data_df = image_data_df.applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158915 entries, 0 to 158914
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   image_name      158915 non-null  object
 1   comment_number  158915 non-null  object
 2   comment         158915 non-null  object
dtypes: object(3)
memory usage: 3.6+ MB


In [151]:
image_data_df.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,Two young
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


## Merge các câu comment trên cùng một ảnh lại với nhau

In [154]:
image_data_df['full_comment'] = image_data_df.groupby('image_name')['comment'].transform(lambda x: ' '.join(x))
image_data_df = image_data_df.drop_duplicates(subset=['image_name'])
image_data_df = image_data_df.drop(['comment_number', 'comment'], axis=1)
image_data_df = image_data_df.reset_index(drop=True)
image_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31783 entries, 0 to 31782
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   image_name    31783 non-null  object
 1   full_comment  31783 non-null  object
dtypes: object(2)
memory usage: 496.7+ KB


In [155]:
image_data_df.head(2)

,image_name,full_comment
0,1000092795.jpg,Two young guys with shaggy hair look at their...
1,10002456.jpg,Several men in hard hats are operating a gian...


## Sau đó, tiến hành check trong câu comment có đề cập con vật nào không? Nếu có thì note lại là có, còn không thì skip

In [157]:
def filter_animal(sentence): # để dành cái này check animal sài
    animal_dictionary_df = pd.read_csv("./animal_dictionary.csv",header=None)
    rows_list = animal_dictionary_df[0].values
    list_has = []
    
    if isinstance(sentence, str):
        # Chuyển câu về chữ thường để so sánh không phân biệt chữ hoa/chữ thường
        sentence_lower = sentence.lower()

        # Kiểm tra xem từng từ trong mảng có xuất hiện trong câu không
        for word in rows_list:
            sentence_word = sentence_lower.split(" ")
            if word.lower() in sentence_word:
                list_has.append(word) # Lưu lại animal đó
                # return True  # Nếu có ít nhất một từ xuất hiện, trả về True
    # else:
    #     print("🌷")
    #     print(type(sentence))
    #     print(sentence)
        
    return list_has
    # return False  # Nếu không có từ nào xuất hiện, trả về False

In [158]:
# Transform dataframe
image_data_df["has_animal"] = image_data_df["full_comment"].apply(lambda x: filter_animal(str(x)))
image_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31783 entries, 0 to 31782
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   image_name    31783 non-null  object
 1   full_comment  31783 non-null  object
 2   has_animal    31783 non-null  object
dtypes: object(3)
memory usage: 745.0+ KB


In [159]:
image_data_df.head(1)

,image_name,full_comment,has_animal
0,1000092795.jpg,Two young guys with shaggy hair look at their...,[]


## Lọc các ảnh có chứa từ khóa

In [163]:
animal_data_df = image_data_df[image_data_df['has_animal'].apply(len) > 0]
animal_data_df.head(5)

,image_name,full_comment,has_animal
6,1000919630.jpg,A man sits in a chair while holding a large s...,"[Animal, Lion]"
12,1001773457.jpg,A black dog and a white dog with brown spots ...,[Dog]
17,1003163366.jpg,a man sleeping on a bench outside with a whit...,[Dog]
29,100759042.jpg,Young man in jacket holding a toothpick with ...,[Crane]
34,1009434119.jpg,A black and white dog is running in a grassy ...,[Dog]


## Tạo folder các con vật

In [172]:
file_df = pd.read_csv("./animal_dictionary.csv", header=None)
file_df.head(1)

,0
0,Aardvark


In [180]:
for index, row in file_df.iterrows():
    folder_name = row.values[0]
    os.makedirs("./animals/" + folder_name, exist_ok=True)
print("Các thư mục đã được tạo thành công!")

Các thư mục đã được tạo thành công!


## Lưu các con vật đã được gán nhãn vào folder tương ứng

In [189]:
for index, row in animal_data_df.iterrows():
    # Lấy giá trị từ cột 'has_animal'
    animals = row['has_animal']
    # Lặp qua từng giá trị trong danh sách
    for animal in animals:
        image_path = os.path.join("./flickr30K_images", row['image_name'])
        if os.path.exists(image_path):
            shutil.copy(image_path, f"./animals/{animal}") 

In [194]:
parent_folder = "./animals"

for folder_name in os.listdir(parent_folder):
    folder_path = os.path.join(parent_folder, folder_name)
    if os.path.isdir(folder_path) and not os.listdir(folder_path):
        os.rmdir(folder_path)
